# Predicting Usage of a Video Game Research Server

## From individual
covers loading data, wrangling and cleaning data, summary of dataset, and exploratory visuals (will need to be changed a bit).

Q: Can total time played predict whether or not they are subscribed to a newsletter

### **Data Description for players.csv:**

**Number of observations**: 196

**Number of variables**: 7

**Names and types of variables:**

- experience: Character (chr)
- subscribe: Logical (lgl)
- hashedEmail: Character (chr)
- played_hours: Double (dbl)
- name: Character (chr)
- gender: Character (chr)
- Age: Double (dbl)

**What does each variable mean?**

- experience: The player's experience level in the game.
- subscribe: Whether or not the player is subscribed to a gaming newsletter.
- hashedEmail: The player's email.
- played_hours: How many hours the player played for.
- name: The real name of the player.
- gender: The gender identity of the player.
- Age: The age of the player.

**Issues with the data**

An issue with the dataset is that there are NA's present. The summary() function was used to create a summary of the data and it was determined that there were 2 NA values in the Age column. These NA values will make it difficult to perform summary statistics on the data set or use it for classification; thus, they must be removed.

**Issues with how the data was collected**

Much of the data appears to be self reported so it is difficult to determine the accuracy of the collected demographic information. One variable in particular that has a large potential to be innacurate is experience. Minecraft does not have a ranking system for players, so the experience levels such as Pro, Veteran, Amateur, etc. are all self reported by the players. As a result, there is potential for their experience level to be exagerated. The other variables also have potential to be innacurately reported as players could potentially lie about their age or input a gender that they do not actually identify with.

In [ ]:
library(tidyverse) 
#This loads in the tidyverse package. This is the main package used in R to help tidy and present data.
#It is neccessary to have this package downloaded for the functions used in this file to run.
set.seed(42)
#This seed makes the project reproducible.

In [ ]:
players <- read_csv("https://raw.githubusercontent.com/pwelters/dsci_project_data/refs/heads/main/players.csv")
#Now the players.csv file is being read into R from a public GitHub repository where it has been downloaded and uploaded to. 
#The dataset will now be visible in this file.
summary(players)

### **Question**

**Can played hours accurately predict whether or not a player is subscribed to a gaming newsletter?**

This question will be addressed using data from the players.csv dataset. I plan to select the data from the subscribe and played_hours columns and use these observations to plot several visualizations that will help to display the data, as well as perform a classification using these observations to determine if played_hours and Age can predict whether or not a player will subscribe to a gaming newsletter with accuracy.

**Tidying Data**

The data from the players dataset is already fairly tidy so it is not neccessary to do much wrangling. However, the Age column has NA values which will make it impossible to compute the mean. To tidy the data, an object called tidy_players was created to remove the NA values from the players dataset. The tidy_players object was then summarized and the mean values for the played_hours and Age columns were computed.

In [ ]:
tidy_players <- drop_na(players)
#drop_na is used to remove the NA values from the players dataset in order to calculate the mean for the quantitative variables.

In [ ]:
players_summary <- tidy_players  |>
summarize(played_hours_mean = mean(played_hours))

players_summary

**Mean value for played_hours**: 5.904 (hours)

### **Visualizations**

In [ ]:
plot_1 <- tidy_players |>
ggplot(aes (x = Age, y = played_hours)) +
geom_point() +
labs(x = "Age of player", y = "Time spent playing (hours)") +
ggtitle("The Relationship between Time Spent Playing and Age")

plot_1

This plot of the relationship between time spent playing and age aims to determine whether or not there is an obvious relationship between the played_hours and Age variables. The graph does not provide evidence of a trend.

In [ ]:
plot_2 <- tidy_players |>
ggplot(aes( x = Age, y = played_hours))+
geom_point(aes(color = subscribe)) +
labs( x = "Age of player", y = "Time spent playing (hours)", color = "Did the player subscribe to a gaming newsletter?")+
ggtitle("The Relationship between Time Spent Playing and Age")

plot_2

This second plot was created to display the same data as the first plot, except this time each individual player is also labelled with whether or not they subscribe to a gaming newsletter. If the player does not subscribe to a newsletter, their subscribe value is FALSE and their point on the scatterplot is coloured orange. If the player does subscribe to a newsletter, their subscribe value is TRUE and their point on the scatterplot is coloured blue. This plot reveals much more about the dataset and provides valuable information that may be helpful in answering the question, "Can played hours and age predict whether or not a player is subscribed to a gaming newsletter?"

There appears to be a relationship between the subscribe variable and Age and played_hours. Newsletter subscriptions and age seem to have a negative relationship as while age increases, less points on the graph show the subscribe value to be TRUE (i.e. less points are blue in colour). Most of the subscribers appear to be within ages 10 to 30. Newsletter subscriptions and time spent playing appear to have a positive relationship, as more blue points appear as time spent playing increases.

These relationships give evidence to suggest that the Age and played_hours variables could be used to predict whether subscribe has a TRUE or FALSE value, and therefore predict whether or not a player is subscribed to a gaming newsletter.

In [ ]:
plot_3 <- tidy_players |>
    ggplot(aes( x = played_hours))+
    geom_histogram(alpha=0.5, position = "identity", aes(color = subscribe)) +
    labs( x = "Time spent playing (hours)", y = "Number of players", color = "Did the player subscribe to a gaming newsletter?")+
    ggtitle("Distribution of Players Based on Their Total Played Hours") +
facet_grid(.~subscribe)

plot_3

### **Method**

To answer the research question, a classifier will be constructed and it will be determined if it is more accurate than simply guessing whether or not someone is subscribed to a newsletter. We can see from the summary of the dataset that there were 144 TRUE responses and 52 false responses. If one were to guess TRUE for every response, they would have an accuracy of 144/196 or 73%. For this classifier to be useful and to prove that age and time played can predict whether or not an individual subscribes to a newsletter, it must have an accuracy greater than 73%.

This method is appropriate as it both creates a classifier to answer one of the questions of interest, and determines whether or not the answers are accurate enough to warrant the use of the classifier.

Some neccessary assumptions are that the players accurately reported their age and that the dataset is large enough to effectively train and test the classifier to the point that it can be trusted to accurately predict the class of an observation for which class is unknown.

A potential limitation of this method is the gaps in collected data for the predictor variables. The majority of the data collected is from players aged 10 to 30 who played under 50 hours. The classifier could potentially have trouble predicting a class for data that falls outside this range as it will not encounter such values as often while being trained.

A classification model was selected for this project as whether or not someone is subscribed to a newsletter is a class variable rather than a numerical value.

To build the classifier, the data being analyzed must first be split into a training set and a test set. The data will be split with 75% of it going into the training set, and 25% into the test set. The training set is used to construct the classifier, which is then used to predict the class of different observations from the test set. Cross validation will be used to select the most accurate k value based on the dataset. Then, the accuracy of the classifier must be tested. To do so, a confusion matrix is generated which provides the values for the total number of correct classifications as well as the total number of all classifications. The metrics of this confusion matrix will then be determined and its accuracy will be calculated. This will give us the accuracy of the classifier. This accuracy value can be compared with the accuracy of just guessing (73%), and it will be determined whether or not the classifier is accurate enough to be deemed useful.

## Data Analysis